In [14]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [15]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
print(train_input.shape, test_input.shape)

(5197, 3) (1300, 3)


In [16]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)
print(sub_input.shape, val_input.shape)


(4157, 3) (1040, 3)


In [17]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.8634615384615385


In [18]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00992537, 0.00712872, 0.00755835, 0.00771046, 0.00697732]), 'score_time': array([0.00106382, 0.00103664, 0.00125766, 0.00108528, 0.00097942]), 'test_score': array([0.87019231, 0.84807692, 0.87969201, 0.85370549, 0.83926853])}


In [19]:
import numpy as np
np.mean(scores.get('test_score'))

0.8581870511586585

In [20]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42))
print(np.mean(scores['test_score']))

0.8578068030235662


In [23]:
from sklearn.model_selection import GridSearchCV
params = {
    "min_impurity_decrease": [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]
}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.9615162593804117
0.8653846153846154


In [26]:
print(gs.cv_results_['mean_test_score'])
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
[{'min_impurity_decrease': 0.0001}, {'min_impurity_decrease': 0.0002}, {'min_impurity_decrease': 0.0003}, {'min_impurity_decrease': 0.0004}, {'min_impurity_decrease': 0.0005}]


In [28]:
params = {
    'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
    'max_depth': range(5, 20, 1),
    'min_samples_split': range(2, 100, 10),
}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [30]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [32]:
from scipy.stats import uniform, randint
rgen = randint(0, 10)
rgen.rvs(10)

array([8, 9, 0, 4, 4, 2, 3, 8, 3, 9])

In [34]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 98,  90,  97, 101, 110, 106, 100, 106, 106,  86]))

In [35]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.57812616, 0.71206463, 0.08414761, 0.17647414, 0.34496259,
       0.20681061, 0.37458537, 0.62288082, 0.15425192, 0.51853911])

In [36]:

params = {
    'min_impurity_decrease': uniform(0.0001, 0.001),
    'max_depth': randint(20, 50),
    'min_samples_split' : randint(2, 25),
    'min_samples_leaf': randint(1, 25),
}

In [37]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(
    DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42
)
gs.fit(train_input, train_target)
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [38]:
print(np.max(gs.cv_results_['mean_test_score']))
dt = gs.best_estimator_
dt.score(test_input, test_target)

0.8695428296438884


0.86